# Code adapted from
* https://dzlab.github.io/dltips/en/pytorch/torchtext-datasets/ to use custom dataset
* https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb to use model

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [2]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  preprocessing = generate_bigrams)

LABEL = data.LabelField(dtype = torch.float)

In [3]:
BATCH_SIZE = 128#64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train, val, test = data.TabularDataset.splits(
    path='./data', train='Train.csv',
    validation='Val.csv', test='Test.csv', format='csv',
    fields=[('text', TEXT),(None, None), ('label', LABEL)],
    skip_header = True)# ignore helpfulness column
    # columns are Text, helpfulness, and rating

train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), batch_sizes=(BATCH_SIZE,BATCH_SIZE,BATCH_SIZE),
    sort_key=lambda x: len(x.text), device = device)

In [4]:
MAX_VOCAB_SIZE = 50_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [6]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

#model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [9]:
"""
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')
"""
print()

In [10]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

NameError: name 'model' is not defined

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(),lr=0.003)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [11]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [12]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
import torch.optim as optim


def train_model(adam=0,lr=0.001,epochs=5):
    # use base adam if adam <=0.5 otherwise use adamW with weight decay
    model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
    optimizer = optim.Adam(model.parameters(),lr=lr) if adam<=0.5 else optim.AdamW(model.parameters(),lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    model = model.to(device)
    criterion = criterion.to(device)
    N_EPOCHS = round(epochs)
    #best_valid_loss = float('inf')
    start_time = time.time()
    for epoch in range(N_EPOCHS):

        

        train_loss, train_acc = train(model, train_iter, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, val_iter, criterion)

       

        
        
       # if valid_loss < best_valid_loss:
       #     best_valid_loss = valid_loss
       #     torch.save(model.state_dict(), 'sentiment.pt')
        
        print(f'Epoch: {epoch+1:02}')# | Epoch Time: {epoch_mins}m {epoch_secs}s'
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Total Train time for {N_EPOCHS} epochs: {epoch_mins}m {epoch_secs}s')
    print("------------------------------------------------------------")
    return -valid_loss
    
    

In [27]:
from bayes_opt import BayesianOptimization
# Bounded region of parameter space
pbounds = {'adam':(0,1),'lr': (0.00001, 0.05), 'epochs': (3, 12)}

optimizer = BayesianOptimization(
    f=train_model,
    pbounds=pbounds,
    random_state=SEED,
)
optimizer.maximize(
    init_points=5,
    n_iter=10,
)

|   iter    |  target   |   adam    |  epochs   |    lr     |
-------------------------------------------------------------
Epoch: 01
	Train Loss: 0.308 | Train Acc: 81.33%
	 Val. Loss: 1.279 |  Val. Acc: 76.31%
Epoch: 02
	Train Loss: 0.210 | Train Acc: 80.72%
	 Val. Loss: 1.743 |  Val. Acc: 74.17%
Epoch: 03
	Train Loss: 0.167 | Train Acc: 80.24%
	 Val. Loss: 1.880 |  Val. Acc: 77.66%
Epoch: 04
	Train Loss: 0.133 | Train Acc: 79.96%
	 Val. Loss: 2.815 |  Val. Acc: 74.64%
Epoch: 05
	Train Loss: 0.100 | Train Acc: 79.61%
	 Val. Loss: 2.885 |  Val. Acc: 76.97%
Epoch: 06
	Train Loss: 0.066 | Train Acc: 79.48%
	 Val. Loss: 4.562 |  Val. Acc: 78.17%
Epoch: 07
	Train Loss: 0.025 | Train Acc: 79.36%
	 Val. Loss: 6.714 |  Val. Acc: 79.72%
Epoch: 08
	Train Loss: -0.032 | Train Acc: 79.29%
	 Val. Loss: 10.629 |  Val. Acc: 75.81%
Epoch: 09
	Train Loss: -0.113 | Train Acc: 79.21%
	 Val. Loss: 14.944 |  Val. Acc: 77.78%
Total Train time for 9 epochs: 16m 2s
------------------------------------------

Epoch: 06
	Train Loss: 0.471 | Train Acc: 82.52%
	 Val. Loss: 0.611 |  Val. Acc: 82.44%
Total Train time for 6 epochs: 10m 49s
------------------------------------------------------------
|  9        | -0.6107   |  0.2745   |  5.952    |  0.000133 |
Epoch: 01
	Train Loss: 0.309 | Train Acc: 81.19%
	 Val. Loss: 1.035 |  Val. Acc: 78.49%
Epoch: 02
	Train Loss: 0.217 | Train Acc: 80.59%
	 Val. Loss: 1.321 |  Val. Acc: 78.39%
Epoch: 03
	Train Loss: 0.173 | Train Acc: 80.16%
	 Val. Loss: 1.562 |  Val. Acc: 75.92%
Epoch: 04
	Train Loss: 0.141 | Train Acc: 79.93%
	 Val. Loss: 2.331 |  Val. Acc: 76.76%
Epoch: 05
	Train Loss: 0.110 | Train Acc: 79.62%
	 Val. Loss: 2.571 |  Val. Acc: 79.10%
Total Train time for 5 epochs: 9m 9s
------------------------------------------------------------
|  10       | -2.571    |  0.1746   |  5.277    |  0.05     |
Epoch: 01
	Train Loss: 0.309 | Train Acc: 81.19%
	 Val. Loss: 1.183 |  Val. Acc: 74.00%
Epoch: 02
	Train Loss: 0.218 | Train Acc: 80.62%
	 Val. Loss: 

In [28]:
print(optimizer.max)
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

{'target': -0.610654134467365, 'params': {'adam': 0.2745473095826844, 'epochs': 5.952006228437811, 'lr': 0.00013297883561406217}}
Iteration 0: 
	{'target': -14.94432977598208, 'params': {'adam': 0.1915194503788923, 'epochs': 8.598978939358487, 'lr': 0.021892009672965652}}
Iteration 1: 
	{'target': -1.597478325667587, 'params': {'adam': 0.7853585837137692, 'epochs': 10.019782273069232, 'lr': 0.013636904338079254}}
Iteration 2: 
	{'target': -28.163462447401823, 'params': {'adam': 0.2764642551430967, 'epochs': 10.216849597815173, 'lr': 0.047907386290648425}}
Iteration 3: 
	{'target': -1.2275178161939944, 'params': {'adam': 0.8759326347420947, 'epochs': 6.2203554296208, 'lr': 0.0250547463249177}}
Iteration 4: 
	{'target': -1.4251740560591704, 'params': {'adam': 0.6834629351721363, 'epochs': 9.414318242846102, 'lr': 0.018518835231971842}}
Iteration 5: 
	{'target': -1.1399453700922628, 'params': {'adam': 0.7809393108395023, 'epochs': 9.31704812255748, 'lr': 0.04597356133128867}}
Iteration 6:

In [29]:
def train_model_save(adam=0,lr=0.001,epochs=5):
    # use base adam if adam <=0.5 otherwise use adamW with weight decay
    model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)
    pretrained_embeddings = TEXT.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
    optimizer = optim.Adam(model.parameters(),lr=lr) if adam<=0.5 else optim.AdamW(model.parameters(),lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    model = model.to(device)
    criterion = criterion.to(device)
    N_EPOCHS = round(epochs)
    start_time = time.time()

    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):

        start_time = time.time()

        train_loss, train_acc = train(model, train_iter, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, val_iter, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'sentiment.pt')

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    return model

In [30]:
model = train_model_save(adam= optimizer.max['params']['adam'], lr=optimizer.max['params']['lr'],
                 epochs=optimizer.max['params']['epochs'])

Epoch: 01 | Epoch Time: 1m 40s
	Train Loss: 0.629 | Train Acc: 82.47%
	 Val. Loss: 0.660 |  Val. Acc: 82.60%
Epoch: 02 | Epoch Time: 1m 42s
	Train Loss: 0.589 | Train Acc: 82.47%
	 Val. Loss: 0.574 |  Val. Acc: 82.60%
Epoch: 03 | Epoch Time: 1m 41s
	Train Loss: 0.554 | Train Acc: 82.47%
	 Val. Loss: 0.551 |  Val. Acc: 83.31%
Epoch: 04 | Epoch Time: 1m 41s
	Train Loss: 0.524 | Train Acc: 82.42%
	 Val. Loss: 0.577 |  Val. Acc: 83.47%
Epoch: 05 | Epoch Time: 1m 40s
	Train Loss: 0.496 | Train Acc: 82.41%
	 Val. Loss: 0.588 |  Val. Acc: 82.81%
Epoch: 06 | Epoch Time: 1m 40s
	Train Loss: 0.469 | Train Acc: 82.57%
	 Val. Loss: 0.598 |  Val. Acc: 82.15%


In [32]:
model.load_state_dict(torch.load('sentiment.pt'))
criterion = nn.BCEWithLogitsLoss()
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.554 | Test Acc: 83.32%


In [33]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [49]:
predict_sentiment(model, "this book was bad")

0.9986447691917419

In [50]:
predict_sentiment(model, "this movie is good")

0.9877820611000061

# Problems
- Some words are not processed correctly: good has high sentiment but so does bad

In [ ]:
import pandas as pd
df= pd.read_csv("Tokenized.csv")
df.head()

In [ ]:
df["Prediction"]= df["Text"].map(lambda x:predict_sentiment(model, x))

In [ ]:
df["Scaled_Prediction"]= df.apply(lambda row:(row['Prediction']*row['Helpfulness']), axis=1)

In [ ]:
df.head()

In [ ]:
df.to_csv("results.csv",index=False)

In [ ]:
df.Rating.value_counts()